# Imports

In [ ]:
from modules.data import  data_reader, data_loader
from modules.sr import result_saver
from modules.models import lag_llama
from modules.experiment.tscv import get_tscv_results, get_summary, extract_metrics
from modules.visualization import graphs
from modules.fine_tuning import lag_llama_ft
from modules.models import prophet


In [26]:
from datetime import datetime, timedelta

today_date = datetime.today().strftime('%Y-%m-%d')
tomorrow_date = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
yesterday_date = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')

start = datetime.strptime(yesterday_date, "%Y-%m-%d")
end = datetime.strptime(tomorrow_date, "%Y-%m-%d")
difference = end - start
print(difference.days == 2)

True


In [36]:
from modules.data import data_loader

a = data_loader.get_data(type = "exchange_rate", ticker = "USD/EUR", frequency = "minutely", start = yesterday_date, end = tomorrow_date)
#a = data_loader.get_data(type = "stock", ticker = "IBM", frequency = "daily", start = "01-01-2023", end = "01-01-2024")

In [37]:
a

,ds,y
0,2024-08-15 15:12:00,0.9099
1,2024-08-15 15:11:00,0.9100
2,2024-08-15 15:10:00,0.9101
3,2024-08-15 15:09:00,0.9101
4,2024-08-15 15:08:00,0.9102
...,...,...
1435,2024-08-14 15:17:00,0.9067
1436,2024-08-14 15:16:00,0.9067
1437,2024-08-14 15:15:00,0.9067
1438,2024-08-14 15:14:00,0.9066


In [ ]:
import os
import requests
keyword = "dowj"
url = f'https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords={keyword}&apikey={os.getenv("ALPHAVANTAGE_API")}'
r = requests.get(url)
data = r.json()

print(data)

{'bestMatches': [{'1. symbol': 'IGAAJX', '2. name': 'DowJon Ttl Market Prt Enhc Ind Strtg 2020/4 USD IRG CASH', '3. type': 'Mutual Fund', '4. region': 'United States', '5. marketOpen': '09:30', '6. marketClose': '16:00', '7. timezone': 'UTC-04', '8. currency': 'USD', '9. matchScore': '0.2000'}]}


In [ ]:
a

,ds,y
0,2024-08-12,81.45
1,2024-08-09,78.28
2,2024-08-08,77.64
3,2024-08-07,76.68
4,2024-08-06,74.60
...,...,...
10068,1986-01-08,25.87
10069,1986-01-07,25.85
10070,1986-01-06,26.53
10071,1986-01-03,26.00


In [ ]:
len(a)

124

In [ ]:
import pandas as pd
full_date_range = pd.date_range(start=a["ds"].min(), end=a["ds"].max(), freq="min")

In [ ]:
full_date_range

DatetimeIndex(['2023-08-31 00:00:00', '2023-08-31 00:01:00',
               '2023-08-31 00:02:00', '2023-08-31 00:03:00',
               '2023-08-31 00:04:00', '2023-08-31 00:05:00',
               '2023-08-31 00:06:00', '2023-08-31 00:07:00',
               '2023-08-31 00:08:00', '2023-08-31 00:09:00',
               ...
               '2023-12-31 23:51:00', '2023-12-31 23:52:00',
               '2023-12-31 23:53:00', '2023-12-31 23:54:00',
               '2023-12-31 23:55:00', '2023-12-31 23:56:00',
               '2023-12-31 23:57:00', '2023-12-31 23:58:00',
               '2023-12-31 23:59:00', '2024-01-01 00:00:00'],
              dtype='datetime64[ns]', length=177121, freq='T')

# Parameters

In [ ]:
# experiment parameters
#constraint: FOLDS x PREDICTION_LENGTH + TRAIN_SIZE !!must not!! be bigger than len(data)

PREDICTION_LENGTH = 1 # currentlz only works for PREDICTION_LENGTH > 1
TICKER = "S&P 500"
FREQUENCY = "minutely" # currently we only have dailz frequency
TYPE_OF_DATA = "stock" # currently we only have stock prices saved
MODELS = ["arima", "llama", "autoregressor", "fine-tuned Llama"] # currentlz works onlz for these two
FOLDS = 5 # for TSCV # reduced to two for testing purposes
CONTEXT_LENGTH = 245 # set to 245 for testing purposes
METRICS = ['r2', 'mse', 'mae', 'rmse', 'mda', "mape"]

# fine-tuning parameters
BATCH_SIZE = 10
MAX_EPOCHS = 5

# data parameters
START_DATE = "2024-07-22"
END_DATE = "2024-07-23"

# want to add
#TRAIN_PERIOD = # context lenghts. Should take a look into this
TRAIN_SIZE = CONTEXT_LENGTH



# Data

In [ ]:
"""
data = data_reader.read_data(type = TYPE_OF_DATA, frequency = FREQUENCY)
# this is just in case there are more CSVs of the same type and frequency, the data should be the first in the list
#if len(data) > 1:
data = data[0]

simple_data = data_reader.read_data(type = TYPE_OF_DATA, frequency = FREQUENCY, match = ["simple"])
#if len(simple_data) > 1:
simple_data = simple_data[0]

train_data = data_reader.read_data(type = TYPE_OF_DATA, frequency = FREQUENCY, match = ["train"])
train_data = train_data[0]

test_data = data_reader.read_data(type = TYPE_OF_DATA, frequency = FREQUENCY, match = ["test"])
test_data = test_data[0]
"""

In [ ]:
DATA_CONFIG = {"ticker" : TICKER,
               "frequency" : FREQUENCY,
               "start" : START_DATE,
               "end" : END_DATE}



In [ ]:
data = data_loader.get_data(data_type=TYPE_OF_DATA, kwargs=DATA_CONFIG)

In [ ]:
data

In [ ]:
import 

# Additional parameters

In [ ]:
DATA_LENGTH = len(data)
FOLDS = int((DATA_LENGTH - TRAIN_SIZE) / PREDICTION_LENGTH) # this calculates max ammount of folds we can have given a set TRAIN_SIZE and PREDICTION_LENGTH
FOLDS = 5


In [ ]:
from modules.data import data_loader

In [ ]:
config = {"ticker" : "USD/GBP",
          "frequency" : "daily",
          "start" : "2024-07-01",
          "end" : "2024-07-30"}

In [ ]:
import os
print(os.getenv("ALPHAVANTAGE_API"))

In [ ]:
os.getenv("ALPHAVANTAGE_API")

In [ ]:
os.environ

In [ ]:
for key, value in os.environ.items():
    print(f'{key}: {value}')

In [ ]:
h = data_loader.get_data(data_type="exchange_rate", kwargs=config)

In [ ]:
h

# Fine tuning

In [ ]:
#preparing the training data for lag llama fine tuning
ft_train_data = lag_llama.prepare_data(data=ft_data, 
                                       prediction_length=0, 
                                       frequency=FREQUENCY)

In [ ]:
# creating the lag llama predictor object 
predictor = lag_llama_ft.get_predictor(prediction_length=PREDICTION_LENGTH, 
                                       context_length=CONTEXT_LENGTH, 
                                       batch_size=BATCH_SIZE, 
                                       max_epochs=MAX_EPOCHS)

In [ ]:
# fine-tuning the predictor object
predictor = predictor.train(ft_train_data, 
                            cache_data = True, 
                            shuffle_buffer_length = 1000)

In [ ]:
r, p = get_tscv_results(data = data,
                           prediction_horizon=PREDICTION_LENGTH,
                           context_length=CONTEXT_LENGTH, 
                           folds=FOLDS, 
                           frequency=FREQUENCY,
                           predictor=predictor)

# Saving results

In [ ]:
r

In [ ]:
p

In [ ]:
experiment_name = f"PREDICTION_LENGTH={PREDICTION_LENGTH}__TICKER={TICKER}__FREQUENCY={FREQUENCY}__TYPE_OF_DATA={TYPE_OF_DATA}__FOLDS={FOLDS}__CONTEXT_LENGTH/TRAIN_SIZE={CONTEXT_LENGTH}__FT_START_DATE={FT_START_DATE}__START_DATE={START_DATE}__END_DATE={END_DATE}__FT_LENGTH={FT_LENGTH}__DATA_LENGTH={DATA_LENGTH}" 

In [ ]:
experiment_name = f"P_L={PREDICTION_LENGTH}__T={TICKER}__FR={FREQUENCY}__T_O_D={TYPE_OF_DATA}__FO={FOLDS}__C_L_T_S={CONTEXT_LENGTH}__FT_S_D={FT_START_DATE}__S_D={START_DATE}__E_D={END_DATE}__FT_L={FT_LENGTH}__D_L={DATA_LENGTH}.csv"

In [ ]:
experiment_name

In [ ]:
result_saver.save_results(r, experiment_name, type="evaluation")
result_saver.save_results(p, experiment_name, type="prediction")

# Models

In [ ]:
# arima


In [ ]:
# lag llama

# Fine tuning an estimator

In [ ]:
PREDICTION_LENGTH = 5

In [ ]:
#preparing the training data for lag llama fine tuning
ft_train_data = lag_llama.prepare_data(data=ft_data, 
                                       prediction_length=0, 
                                       frequency=FREQUENCY)

In [ ]:
# creating the lag llama predictor object 
predictor = lag_llama_ft.get_predictor(prediction_length=PREDICTION_LENGTH, 
                                       context_length=CONTEXT_LENGTH, 
                                       batch_size=BATCH_SIZE, 
                                       max_epochs=MAX_EPOCHS)

In [ ]:
# fine-tuning the predictor object
predictor = predictor.train(ft_train_data, 
                            cache_data = True, 
                            shuffle_buffer_length = 1000)

# SKLearn TSCV

In [ ]:
r, p, a = get_tscv_results(data = data,
                           prediction_horizon=PREDICTION_LENGTH,
                           context_length=CONTEXT_LENGTH, 
                           folds=FOLDS, 
                           frequency=FREQUENCY, 
                           predictor=predictor)

In [ ]:
#creating the summary for each model
s = [get_summary(r[i]) for i in range(len(r))]

In [ ]:
means_df, medians_df, stds_df = extract_metrics(s, MODELS)

In [ ]:
means_df

# Visualisation

In [ ]:
graphs.standard_visualisation(MODELS, METRICS, s)

In [ ]:
graphs.interactive_visualisation(MODELS, METRICS, s)

In [ ]:
graphs.prediction_visualisation(MODELS, p, a)

LABEL the graph
label the axes with timestamps
show the history of the actual
table of comparison, not just graph